<a href="https://colab.research.google.com/github/Utsav147/Google-Colab-TF-Practice/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("hello")

hello


In [5]:
import numpy

In [6]:
import tensorflow as tf
print(tf.version)

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.12/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [10]:
rank1_tensor=tf.Variable([["Test","ok"],["Test","ok"],["Test","ok"]],tf.string)
rank1_tensor.shape

TensorShape([3, 2])